In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

In [7]:
X,y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=2, n_repeated=0, n_classes=2)

In [8]:
data=pd.DataFrame(X,columns=['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10'])
data['y']=y

In [9]:
data

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,y
0,1.111494,0.495560,-0.414707,-0.384336,-1.013636,1.389230,-0.368272,0.286233,2.061779,0.494246,1
1,-1.326984,0.494637,-0.892782,-0.328157,-2.680694,-0.000455,-0.179112,-3.000701,0.886162,-1.033059,1
2,-0.045274,0.293912,-0.014475,1.909269,-4.154936,-3.050023,-1.284813,1.267961,-0.641287,-5.941790,0
3,0.747838,0.110884,-0.792524,-1.236409,-1.881237,0.534894,0.143366,0.011084,3.300637,-0.876590,1
4,-1.132118,0.218735,-0.052822,1.030900,-2.176248,-1.268790,-0.260134,-1.318788,1.926086,-2.825124,1
...,...,...,...,...,...,...,...,...,...,...,...
995,0.464577,1.034793,-0.779034,0.370192,0.539478,2.077466,-0.992319,-1.024690,2.568832,2.581464,0
996,1.588742,0.096891,-0.895447,-1.349205,-1.758423,0.120859,1.632216,1.620928,-0.729505,-0.728791,1
997,-3.481468,-0.138043,0.781044,-0.360495,-1.827935,-3.704529,0.078881,-2.722562,0.224309,-4.923787,1
998,0.267597,0.914811,-1.821396,-0.574849,-0.404745,-1.596565,0.499973,0.793549,-1.497834,-0.646848,0


# Creating the Pytorch Dataset

In [78]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,data):
        self.data=data
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        return torch.tensor(self.data.iloc[idx,:-1],dtype=torch.float),torch.tensor(self.data.iloc[idx,-1],dtype=torch.float)

In [79]:
from sklearn.model_selection import train_test_split
train,val=train_test_split(data,test_size=0.2,random_state=42)

In [80]:
train_dataset=Dataset(train)
val_dataset=Dataset(val)

In [81]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader=torch.utils.data.DataLoader(val_dataset,batch_size=32,shuffle=False)

In [100]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.linear1=torch.nn.Linear(10,100)
        self.linear2=torch.nn.Linear(100,100)
        self.linear3=torch.nn.Linear(100,1)
        self.sigmoid=torch.nn.Sigmoid()
    def forward(self,x):
        x=self.linear1(x)
        x=self.linear2(x)
        x=self.linear3(x)
        x=self.sigmoid(x)
        return x

In [101]:
from sklearn.metrics import accuracy_score

In [102]:
# model

In [103]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

# Define the training function
def train(model, train_loader, val_loader, epochs):
    optimizer = torch.optim.Adadelta(model.parameters())
    criterion = torch.nn.BCELoss()

    for epoch in range(epochs):
        total_train_loss = 0
        total_val_loss = 0
        total_train_acc = 0
        total_val_acc = 0
        total_train_samples = 0
        total_val_samples = 0

        model.train()
        for x, y in train_loader:
            optimizer.zero_grad()
            y_pred = model(x)
            y_pred = y_pred.squeeze()  # Squeeze the model output to match the target shape
            train_loss = criterion(y_pred, y)
            train_loss.backward()
            optimizer.step()

            total_train_loss += train_loss.item() * x.size(0)
            train_acc = accuracy_score(y.detach().numpy(), y_pred.detach().round().numpy())
            total_train_acc += train_acc * x.size(0)
            total_train_samples += x.size(0)

        model.eval()
        with torch.no_grad():
            for x, y in val_loader:
                y_pred = model(x)
                y_pred = y_pred.squeeze()  # Squeeze the model output to match the target shape
                val_loss = criterion(y_pred, y)

                total_val_loss += val_loss.item() * x.size(0)
                val_acc = accuracy_score(y.detach().numpy(), y_pred.detach().round().numpy())
                total_val_acc += val_acc * x.size(0)
                total_val_samples += x.size(0)

        avg_train_loss = total_train_loss / total_train_samples
        avg_val_loss = total_val_loss / total_val_samples
        avg_train_acc = total_train_acc / total_train_samples
        avg_val_acc = total_val_acc / total_val_samples

        print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss}, Train Acc: {avg_train_acc}, Val Loss: {avg_val_loss}, Val Acc: {avg_val_acc}')


In [ ]:

model = Model()
train(model, train_loader, val_loader, 100)


Epoch 1, Train Loss: 0.6609921479225158, Train Acc: 0.62375, Val Loss: 0.6703002238273621, Val Acc: 0.67
Epoch 2, Train Loss: 0.6456390047073364, Train Acc: 0.63875, Val Loss: 0.6599378395080566, Val Acc: 0.65
Epoch 3, Train Loss: 0.6506894707679749, Train Acc: 0.635, Val Loss: 0.6555548548698426, Val Acc: 0.645
Epoch 4, Train Loss: 0.6422505927085876, Train Acc: 0.64, Val Loss: 0.6608861994743347, Val Acc: 0.645
Epoch 5, Train Loss: 0.6429236364364624, Train Acc: 0.655, Val Loss: 0.6557884645462037, Val Acc: 0.67
Epoch 6, Train Loss: 0.6369238495826721, Train Acc: 0.63875, Val Loss: 0.6504357314109802, Val Acc: 0.67
Epoch 7, Train Loss: 0.6362177991867065, Train Acc: 0.6625, Val Loss: 0.6506583714485168, Val Acc: 0.665
Epoch 8, Train Loss: 0.6340010809898377, Train Acc: 0.64125, Val Loss: 0.6482849431037903, Val Acc: 0.66
Epoch 9, Train Loss: 0.6382848811149597, Train Acc: 0.64125, Val Loss: 0.6587842726707458, Val Acc: 0.66
Epoch 10, Train Loss: 0.6355349326133728, Train Acc: 0.65375